In [12]:
import pulp

## Input variables 

classes = ['1a', '1b', '1c', '1d', '1e']

courses = ['English', 'German', 'Dutch', 'Math', 'Biology', 'Mentorship hour']

teachers = ['Bob', 'Thea', 'Johan', 'Hanneke', 'Kim', 'Martin', 'Sarah']

classrooms = ['0.1', '0.2', '1.1', '1.2', '1.3', '2.2', '2.3']

hours = [1, 2, 3, 4, 5, 6, 7, 8]


## Parameters

# shows for each teacher which courses he/she is able to give
able_to_give_course = [[1,1,0,0,0,1], [1,0,1,0,0,1],[0,0,0,1,0,1],[0,0,0,0,1,0], [0,0,1,0,0,0],[1,0,0,0,0,1], [0,0,1,1,0,0]]

# shows how many hours of each course each class needs
required_hours = [[2,1,2,2,1,0], [2,2,2,0,0,0], [1,1,1,1,1,1], [1,0,1,2,2,1], [1,0,1,2,2,2]]


## Decision variable

X = pulp.LpVariable.dicts(
    "X", 
    ((h, c, t, r, l)
     for h in hours 
     for c in classes 
     for t in teachers 
     for r in classrooms 
     for l in courses),
    cat='Binary'
)
# this variable equals 1 if course is given at hour h, to class c, by teacher t in class room r, else 0.


## objective

prob = pulp.LpProblem("School_Timetable", pulp.LpMaximize)
prob += pulp.lpSum(X[h, c, t, r, l] for h in hours for c in classes for t in teachers for r in classrooms for l in courses)


## constraints

# No teacher can be double-booked
for h in hours:
    for t in teachers:
        prob += (
            pulp.lpSum(X[h, c, t, r, l] 
                       for c in classes 
                       for r in classrooms 
                       for l in courses)
            <= 1
        )

# No class can have two lessons at the same time
for h in hours:
    for c in classes:
        prob += (
            pulp.lpSum(X[h, c, t, r, l] 
                       for t in teachers 
                       for r in classrooms 
                       for l in courses)
            <= 1
        )

# No room can be double-booked
for h in hours:
    for r in classrooms:
        prob += (
            pulp.lpSum(X[h, c, t, r, l]
                       for c in classes
                       for t in teachers
                       for l in courses)
            <= 1
        )

# Required teaching hours for each class & course
for c_idx, c in enumerate(classes):
    for co_idx, l in enumerate(courses):
        required = required_hours[c_idx][co_idx]
        prob += (
            pulp.lpSum(X[h, c, t, r, l] 
                       for h in hours 
                       for t in teachers 
                       for r in classrooms)
            == required
        )

# A teacher can only teach a course if allowed
for t_idx, t in enumerate(teachers):
    for co_idx, l in enumerate(courses):
        if able_to_give_course[t_idx][co_idx] == 0:
            for h in hours:
                for c in classes:
                    for r in classrooms:
                        prob += X[h, c, t, r, l] == 0


## Solving the problem
prob.solve()
print("Status:", pulp.LpStatus[prob.status])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/yvonne.smeenge/Documents/GitHub/School_timetable_model/.venv/lib/python3.11/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/sk/k2jvbc391_d47y527rssnps00000gp/T/e0d0c4d6258d4d1dafda6a83cbc84e40-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/sk/k2jvbc391_d47y527rssnps00000gp/T/e0d0c4d6258d4d1dafda6a83cbc84e40-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8027 COLUMNS
At line 98188 RHS
At line 106211 BOUNDS
At line 117972 ENDATA
Problem MODEL has 8022 rows, 11760 columns and 54880 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 35 - 0.01 seconds
Cgl0002I 8568 variables fixed
Cgl0004I processed model has 176 rows, 3192 columns (3192 integer (3192 of which binary)) and 12768 elements
Cutoff increment increased from 1e-05 to 0.9999
C

In [15]:
import pandas as pd

schedule = []

for h in hours:
    for c in classes:
        for t in teachers:
            for r in classrooms:
                for l in courses:
                    if pulp.value(X[h, c, t, r, l]) == 1:
                        schedule.append({
                            "Hour": h,
                            "Class": c,
                            "Course": l,
                            "Teacher": t,
                            "Room": r
                        })

df = pd.DataFrame(schedule)
df = df.sort_values(by=["Class", "Hour"])

df

,Hour,Class,Course,Teacher,Room
0,1,1a,German,Bob,0.1
4,2,1a,Dutch,Kim,2.3
8,3,1a,English,Thea,1.1
13,4,1a,English,Martin,1.3
18,5,1a,Math,Johan,2.2
22,6,1a,Biology,Hanneke,1.1
27,7,1a,Math,Sarah,1.2
31,8,1a,Dutch,Kim,1.2
1,1,1b,English,Martin,2.3
5,2,1b,Dutch,Thea,1.1
